<a href="https://colab.research.google.com/github/kayamui/istanbul_analysis/blob/main/extra_shift_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import warnings
warnings.filterwarnings('ignore')

In [2]:
path= 'C:/Users/mkaya/Onedrive/Masaüstü/istanbul112_hidden/finished_works/staff_coordination/'

*/
Necessary files:
1. The all data in 112 online
2. The taken shifts data on 112 online

In [3]:
df_all = pd.read_excel('C:/Users/mkaya/Downloads/OCAK 2024 A BÖLGESİ.xlsx') #The all data, not only the region A

In [4]:
df_all= df_all[~(df_all['Birim Adı'].isin(['AVRUPA KKM','BAHÇELİEVLER DEVLET HASTANESİ 112','BAKIRKÖY MURAT DİLMENER','KKM HAVUZ']))]

In [5]:
df_all['Doktor/AAABT']= df_all['Doktor/AAABT'].astype(str).str.strip().astype(str).str.split(',')
df_all['Att']= df_all['Att'].astype(str).str.strip().astype(str).str.split(',')
df_all['Şoför']= df_all['Şoför'].astype(str).str.strip().astype(str).str.split(',')

In [6]:
df_all['Sağlık']= df_all['Doktor/AAABT'] + df_all['Att']

In [7]:
df_all.drop(columns=['Doktor/AAABT', 'Att'], inplace=True)

In [8]:
health_staff= df_all[['Birim Adı', 'Sağlık']]
driver= df_all[['Birim Adı', 'Şoför']]

In [9]:
health_staff= health_staff.explode('Sağlık')
driver= driver.explode('Şoför')

In [10]:
health_staff= health_staff[health_staff['Sağlık']!='nan']
driver= driver[driver['Şoför']!='nan']

In [11]:
health_staff['Sağlık']= health_staff['Sağlık'].astype(int)
driver['Şoför']= driver['Şoför'].astype(int)

In [12]:
driver['value']= 1
health_staff['value']= 1

In [13]:
df_saglik= health_staff.pivot_table(index='Birim Adı', columns='Sağlık', values='value', aggfunc='sum').fillna(0)
df_driver= driver.pivot_table(index='Birim Adı', columns='Şoför', values='value', aggfunc='sum').fillna(0)

In [14]:
df_saglik['Nokta Toplamı'] = df_saglik.sum(axis=1, numeric_only=True)
df_saglik.loc[len(df_saglik)] = df_saglik.sum(axis=0, numeric_only=True)

In [15]:
df_saglik.rename(index={df_saglik.index[-1]: 'Gün Toplamı'}, inplace=True)

In [16]:
df_driver['Nokta Toplamı'] = df_driver.sum(axis=1, numeric_only=True)
df_driver.loc[len(df_driver)] = df_driver.sum(axis=0, numeric_only=True)
df_driver.rename(index={df_driver.index[-1]: 'Gün Toplamı'}, inplace=True)

In [17]:
df_taken= pd.read_excel('C:/Users/mkaya/Downloads/Alınan_Nöbetler OCAK 2024.xlsx')

In [18]:
df_taken['Nöbet Başlangiç Bitiş Tarihi']= df_taken['Nöbet Başlangiç Bitiş Tarihi'].astype(str).str.split('-')
df_taken['Nöbet Başlangiç Bitiş Tarihi']= df_taken['Nöbet Başlangiç Bitiş Tarihi'].apply(lambda x: x[0])
df_taken['Nöbet Başlangiç Bitiş Tarihi']= df_taken['Nöbet Başlangiç Bitiş Tarihi'].astype(str).str.split(' ')
df_taken['Nöbet Başlangiç Bitiş Tarihi']= df_taken['Nöbet Başlangiç Bitiş Tarihi'].apply(lambda x: x[0])
df_taken['Nöbet Başlangiç Bitiş Tarihi']= df_taken['Nöbet Başlangiç Bitiş Tarihi'].astype(str).str.strip()

In [19]:
df_taken['Nöbet Başlangiç Bitiş Tarihi'] = pd.to_datetime(df_taken['Nöbet Başlangiç Bitiş Tarihi'], format='%d.%m.%Y')

In [20]:
month = df_taken.iloc[0]['Nöbet Başlangiç Bitiş Tarihi'].month
year= df_taken.iloc[0]['Nöbet Başlangiç Bitiş Tarihi'].year

In [21]:
df_saglik.to_excel(path+f'{month}-{year} Açılan Sağlık Nöbeti.xlsx')

In [22]:
df_driver.to_excel(path+f'{month}-{year} Açılan Sürücü Nöbeti.xlsx')

In [23]:
df_taken['Nöbet Başlangiç Bitiş Tarihi']= df_taken['Nöbet Başlangiç Bitiş Tarihi'].dt.day

In [24]:
df_taken['count']= 1

In [25]:
df_taken['Birim Adı']= df_taken['Birim Adı'].astype(str).str.replace('i','İ').astype(str).str.strip().astype(str).str.upper()

In [26]:
df_taken= df_taken[~(df_taken['Birim Adı'].isin(['AVRUPA KKM','BAHÇELİEVLER DEVLET HASTANESİ 112','BAKIRKÖY MURAT DİLMENER','KKM HAVUZ']))]

In [27]:
df_taken_health= df_taken[(df_taken['Nöbet Tipi']=='ATT') |(df_taken['Nöbet Tipi']=='Dr/AABT')]
df_taken_driver= df_taken[df_taken['Nöbet Tipi']=='Sürücü']

In [28]:
taken_health_pivot= df_taken_health.pivot_table(index='Birim Adı', columns='Nöbet Başlangiç Bitiş Tarihi', values='count', aggfunc='sum').fillna(0)
taken_driver_pivot= df_taken_driver.pivot_table(index='Birim Adı', columns='Nöbet Başlangiç Bitiş Tarihi', values='count', aggfunc='sum').fillna(0)

In [29]:
taken_health_pivot['Nokta Toplamı']= taken_health_pivot.sum(axis=1)
taken_driver_pivot['Nokta Toplamı']= taken_driver_pivot.sum(axis=1)
taken_health_pivot.loc[len(taken_health_pivot)]= taken_health_pivot.sum(axis=0)
taken_driver_pivot.loc[len(taken_driver_pivot)]= taken_driver_pivot.sum(axis=0)

In [30]:
taken_health_pivot = taken_health_pivot.rename(index={taken_health_pivot.index[-1]: 'Gün Toplamı'})
taken_driver_pivot = taken_driver_pivot.rename(index={taken_driver_pivot.index[-1]: 'Gün Toplamı'})

In [31]:
taken_health_pivot.to_excel(path+f'{month}-{year} Alınan Sağlık Nöbeti.xlsx')
taken_driver_pivot.to_excel(path+f'{month}-{year} Alınan Sürücü Nöbeti.xlsx')

In [32]:
health_difference_index= df_saglik.index.difference(taken_health_pivot.index)

In [33]:
for index in health_difference_index:
    taken_health_pivot.loc[index]= 0

In [34]:
driver_difference_index= df_driver.index.difference(taken_driver_pivot.index)
for index in driver_difference_index:
    taken_driver_pivot.loc[index]= 0

In [35]:
untaken_health_pivot= df_saglik-taken_health_pivot

In [36]:
untaken_driver_pivot= df_driver - taken_driver_pivot

In [37]:
untaken_driver_pivot.drop(index='Gün Toplamı', inplace=True)
untaken_health_pivot.drop(index='Gün Toplamı', inplace=True)

In [38]:
untaken_driver_pivot.loc[len(untaken_driver_pivot)]= untaken_driver_pivot.sum(axis=0)
untaken_health_pivot.loc[len(untaken_health_pivot)]= untaken_health_pivot.sum(axis=0)

In [39]:
untaken_driver_pivot.rename(index={untaken_driver_pivot.index[-1]: 'Gün Toplamı'}, inplace=True)
untaken_health_pivot.rename(index={untaken_health_pivot.index[-1]: 'Gün Toplamı'}, inplace=True)

In [40]:
untaken_driver_pivot.to_excel(path+f'{month}-{year} Alınmayan Sürücü Nöbeti.xlsx')

In [41]:
untaken_health_pivot.to_excel(path+f'{month}-{year} Alınmayan Sağlık Nöbeti.xlsx')

In [42]:
file_path_1= path+f'{month}-{year} Açılan Sağlık Nöbeti.xlsx'
file_path_2= path+f'{month}-{year} Açılan Sürücü Nöbeti.xlsx'
file_path_3= path+f'{month}-{year} Alınan Sağlık Nöbeti.xlsx'
file_path_4= path+f'{month}-{year} Alınan Sürücü Nöbeti.xlsx'
file_path_5= path+f'{month}-{year} Alınmayan Sağlık Nöbeti.xlsx'
file_path_6= path+f'{month}-{year} Alınmayan Sürücü Nöbeti.xlsx'

In [43]:
from openpyxl import load_workbook
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.utils import get_column_letter

file_paths= [file_path_1,file_path_2,file_path_3,file_path_4,file_path_5,file_path_6]
for file_path in file_paths:

  # Load the workbook and select the active worksheet
  wb = load_workbook(file_path)
  ws = wb.active

  # Find last row and column
  max_row = ws.max_row
  max_col = ws.max_column

  # Apply conditional formatting for "Nokta Toplamı" (last column, excluding last row)
  nokta_toplami_col_letter = get_column_letter(max_col)
  nokta_toplami_range = f"{nokta_toplami_col_letter}2:{nokta_toplami_col_letter}{max_row-1}"

  color_scale_rule_nokta = ColorScaleRule(
      start_type="min", start_color="FFFFFF",
      mid_type="percentile", mid_value=50, mid_color="FFFF99",
      end_type="max", end_color="FF0000"
  )
  ws.conditional_formatting.add(nokta_toplami_range, color_scale_rule_nokta)

  # Apply conditional formatting for "Gün Toplamı" (last row, excluding last column)
  gun_toplami_row = max_row
  gun_toplami_range = f"B{gun_toplami_row}:{get_column_letter(max_col - 1)}{gun_toplami_row}"

  color_scale_rule_gun = ColorScaleRule(
      start_type="min", start_color="FFFFFF",
      mid_type="percentile", mid_value=50, mid_color="FFFF99",
      end_type="max", end_color="FF0000"
  )
  ws.conditional_formatting.add(gun_toplami_range, color_scale_rule_gun)

  # Save the modified file
  formatted_file_path = file_path
  wb.save(formatted_file_path)

  print(f"Formatted file saved as {formatted_file_path}")


Formatted file saved as C:/Users/mkaya/Onedrive/Masaüstü/istanbul112_hidden/finished_works/staff_coordination/1-2024 Açılan Sağlık Nöbeti.xlsx
Formatted file saved as C:/Users/mkaya/Onedrive/Masaüstü/istanbul112_hidden/finished_works/staff_coordination/1-2024 Açılan Sürücü Nöbeti.xlsx
Formatted file saved as C:/Users/mkaya/Onedrive/Masaüstü/istanbul112_hidden/finished_works/staff_coordination/1-2024 Alınan Sağlık Nöbeti.xlsx
Formatted file saved as C:/Users/mkaya/Onedrive/Masaüstü/istanbul112_hidden/finished_works/staff_coordination/1-2024 Alınan Sürücü Nöbeti.xlsx
Formatted file saved as C:/Users/mkaya/Onedrive/Masaüstü/istanbul112_hidden/finished_works/staff_coordination/1-2024 Alınmayan Sağlık Nöbeti.xlsx
Formatted file saved as C:/Users/mkaya/Onedrive/Masaüstü/istanbul112_hidden/finished_works/staff_coordination/1-2024 Alınmayan Sürücü Nöbeti.xlsx
